<a href="https://colab.research.google.com/github/shizoda/education/blob/main/image/dcgan_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 DCGAN による画像生成（CIFAR-10）

このノートブックでは、 **深層畳み込み生成敵対ネットワーク（DCGAN: Deep Convolutional Generative Adversarial Networks）** を用いて、CIFAR-10 データセットの画像生成を行います。

## 🔍 従来の畳み込みアルゴリズムとの比較

DCGAN を理解する上では、CNN や U-Net とのアーキテクチャおよび出力の目的の違いに注目しましょう。

### 1. CNN（画像分類）
**畳み込み層（Convolution）** を用いて画像から特徴を抽出し、最終的にそれが何であるかというクラスラベルを出力します。画像生成とは逆の「情報の圧縮」を行うプロセスです。

*  [PyTorch による CNN 画像分類（CIFAR-10）](https://github.com/shizoda/education/blob/ae8526e388ed2f2e05eb5209db51340d40f46d68/machine_learning/cnn/cifar10_pytorch.ipynb)

### 2. U-Net（画像セグメンテーション）
エンコーダで特徴を抽出した後、 **転置畳み込み（Transposed Convolution）** を用いて特徴量から元の画像と同じ解像度の **確率マップ** を出力します。これにより「どのピクセルがどの物体に属するか」を特定します。

*  [U-Net による肺がん領域のセグメンテーション](https://github.com/shizoda/education/blob/ae8526e388ed2f2e05eb5209db51340d40f46d68/machine_learning/unet/unet_lung_cancer.ipynb)

### 3. DCGAN（画像生成）
DCGAN の生成器（Generator）は、U-Net のデコーダと同様に **転置畳み込み層** を活用しますが、その目的が大きく異なります。U-Net が特徴量から確率マップを復元するのに対し、DCGAN は **ランダムなノイズ（潜在変数 $z$）** を入力として、そこから **本物らしい画像そのもの（Fake画像）** を出力します。

| 手法 | 入力 | 主要技術 | 出力 |
| :--- | :--- | :--- | :--- |
| **CNN** | 画像 | 畳み込み | クラスラベル |
| **DCGAN の Discriminator (分類器)** | 画像 | 畳み込み | Real / Fake 判定 （Real である確率）|
| **U-Net** | 画像 | 畳み込み + 転置畳み込み | 確率マップ |
| **DCGAN の Generator (生成器)** | ノイズ | 転置畳み込み | 生成画像（Fake） |

## 📐 ネットワーク構造の詳細

DCGAN のネットワーク構造を詳細に解説します。よくわからない場合はいったんスキップして、コードの実行に進んでください｡

### 1. Generator (Decoupling and Expansion)
Generator は **U-Net** のデコーダ部と同様に **Transposed Convolution（転置畳み込み）** を使用します。 U-Net が「特徴量からセグメンテーション用の確率マップ」を復元するのに対し、 DCGAN は 100 次元の潜在変数 $z$ から $32 \times 32$ の **Fake Image（画像そのもの）** を生成します。

* **Spatial Transformation**: カーネルサイズ 4、ストライド 2、パディング 1 の転置畳み込みにより、解像度を $4 \rightarrow 8 \rightarrow 16 \rightarrow 32$ へと段階的に拡大します。
* **Activation**: 各層で **Batch Normalization** と **ReLU** を適用し、最終層のみ **Tanh** を用いて画素値を $[-1, 1]$ に収めます。

### 2. Discriminator (Feature Extraction and Compression)
Discriminator は典型的な **CNN** の構造を持ちます。通常の画像分類器が「画像 $\rightarrow$ 多クラスラベル」を出力するのに対し、ここでは「画像 $\rightarrow$ 1つのスカラー値（Real である確率）」へと情報を圧縮します。

* **Spatial Transformation**: 畳み込みにより解像度を $32 \rightarrow 16 \rightarrow 8 \rightarrow 4$ と縮小していきます。
* **Activation**: 勾配消失を防ぐために **LeakyReLU** を使用し、最終層で **Sigmoid** を適用して確率値を出力します。

### 3. DCGAN Connectivity
システム全体としては、 Generator が生成した Fake 画像と、 CIFAR-10 の Real 画像が Discriminator に入力される構成となります。学習が進むにつれ、 Generator は Discriminator の判断を惑わせるほど精巧な画像を生成するようになります。

In [21]:
# @title 📊 Visualize Network Structure
# @markdown 表示するネットワークの構成を選択してください。
display_target = "dcgan_combined" # @param ["generator_detail", "discriminator_detail", "dcgan_combined"]

import base64
from IPython.display import Image, display

def render_mermaid(graph_code):
    """
    Renders Mermaid diagram using mermaid.ink API.
    Supports UTF-8 characters and renders high-quality PNG.
    """
    graphbytes = graph_code.encode("utf-8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

# Define Mermaid codes for each component
graphs = {
    "generator_detail": """
graph TD
    Z[Input Noise: 100x1x1] --> G1[ConvTranspose2d: k4 s1 p0]
    G1 --> G1_Out[Feature Map: 512x4x4]
    G1_Out --> G1_Act[BN / ReLU]
    G1_Act --> G2[ConvTranspose2d: k4 s2 p1]
    G2 --> G2_Out[Feature Map: 256x8x8]
    G2_Out --> G2_Act[BN / ReLU]
    G2_Act --> G3[ConvTranspose2d: k4 s2 p1]
    G3 --> G3_Out[Feature Map: 128x16x16]
    G3_Out --> G3_Act[BN / ReLU]
    G3_Act --> G4[ConvTranspose2d: k4 s2 p1]
    G4 --> G4_Out[Output Image: 3x32x32]
    G4_Out --> G_Tanh[Tanh Activation]
    """,

    "discriminator_detail": """
graph TD
    Input[Input Image: 3x32x32] --> D1[Conv2d: k4 s2 p1]
    D1 --> D1_Out[Feature Map: 64x16x16]
    D1_Out --> D1_Act[LeakyReLU]
    D1_Act --> D2[Conv2d: k4 s2 p1]
    D2 --> D2_Out[Feature Map: 128x8x8]
    D2_Out --> D2_Act[BN / LeakyReLU]
    D2_Act --> D3[Conv2d: k4 s2 p1]
    D3 --> D3_Out[Feature Map: 256x4x4]
    D3_Out --> D3_Act[BN / LeakyReLU]
    D3_Act --> D4[Conv2d: k4 s1 p0]
    D4 --> D4_Out[Logit: 1x1x1]
    D4_Out --> D_Sig[Sigmoid Activation]
    """,

    "dcgan_combined": """
graph LR
    subgraph DCGAN_Connectivity [DCGAN System Structure]
        Z[Latent Noise z] --> G[Generator]
        G -- Fake Image --> D
        Real[Real Image: CIFAR-10] --> D[Discriminator]
        D --> Prob[Output: Probability]
    end
    """
}

print(f"Displaying: {display_target}")
render_mermaid(graphs[display_target])

Displaying: dcgan_combined


## 🛠️ ライブラリの読み込み・ハイパーパラメータの設定


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
import numpy as np

# ハイパーパラメータ
params = {
    "n_epochs": 300,      # エポック数
    "batch_size": 128,    # バッチサイズ
    "lr": 0.0002,         # 学習率
    "beta1": 0.5,         # Adamオプティマイザのパラメータ
    "nz": 100,            # 潜在変数の次元数 (ノイズのサイズ)
    "ngf": 128,            # Generatorの特徴マップのサイズ
    "ndf": 64,            # Discriminatorの特徴マップのサイズ
    "image_size": 32,     # CIFAR-10の画像サイズは32x32
    "nc": 3,              # 画像のチャンネル数 (CIFAR-10はカラーなので3)
    "target_class_name": "automobile" # 学習したいクラス名
}

# GPUが利用可能か確認し、デバイスを設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 🛠️ データセットの準備と正規化

ここでは、CIFAR-10 データセットをロードし、特定のクラス（今回は **automobile** ）のみを抽出して学習データとして使用します。

DCGAN の学習においては、Generator の最終層に `Tanh` 関数を使用することが多いためく、画像データを **[-1, 1]** の範囲に正規化することが一般的です。

In [ ]:
# 画像の前処理
transform = transforms.Compose([
    transforms.Resize(params["image_size"]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # [-1, 1]に正規化
])

# データセットのダウンロード
full_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

# CIFAR-10のクラス名
class_names = full_dataset.classes
print(f"CIFAR-10 class names: {class_names}")

# 学習対象のクラスのインデックスを取得
try:
    target_class_idx = class_names.index(params["target_class_name"])
    print(f"Targeting class: '{params['target_class_name']}' (index: {target_class_idx})")
except ValueError:
    print(f"Error: '{params['target_class_name']}' is not a valid CIFAR-10 class name. Please choose from {class_names}")
    exit()

# 特定のクラスのデータのみを抽出
indices = [i for i, label in enumerate(full_dataset.targets) if label == target_class_idx]
dataset = Subset(full_dataset, indices)
dataloader = DataLoader(dataset, batch_size=params["batch_size"], shuffle=True)

if len(dataset) == 0:
    print(f"Error: No images found for class '{params['target_class_name']}'. Please check the class name or dataset.")
    exit()
else:
    print(f"Number of images for '{params['target_class_name']}': {len(dataset)}")

# データの可視化 (確認用)
real_batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title(f"Training Images ({params['target_class_name']})")
plt.imshow(np.transpose(make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

## 🏗️ ネットワークモデルの定義

DCGAN は **生成器（Generator）** と **識別器（Discriminator）** の 2 つのネットワークが互いに競い合うことで学習が進みます。

### 生成器（Generator）
生成器の役割は、100次元などの低次元のベクトル（ノイズ）を、本物の画像に見える多次元のデータへと変換することです。これには **転置畳み込み（ConvTranspose2d）** を繰り返し使用し、解像度を段階的に拡大（Up-sampling）していきます。

### 生成器（Generator）の実装
生成器は、入力された潜在変数 $z$（100次元のベクトル）を、段階的な転置畳み込みによって $32 \times 32$ ピクセルのカラー画像（3チャンネル）へと変換します。各層の後には **バッチ正規化（Batch Normalization）** を適用し、学習を安定させます。最終層には `Tanh` 関数を用い、出力を **[-1, 1]** の範囲に収めます。

In [ ]:
class Generator(nn.Module):
    def __init__(self, params):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 入力: 潜在変数 z
            nn.ConvTranspose2d(params["nz"], params["ngf"] * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(params["ngf"] * 4),
            nn.ReLU(True),
            # 特徴マップサイズ: (ngf*4) x 4 x 4
            nn.ConvTranspose2d(params["ngf"] * 4, params["ngf"] * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(params["ngf"] * 2),
            nn.ReLU(True),
            # 特徴マップサイズ: (ngf*2) x 8 x 8
            nn.ConvTranspose2d(params["ngf"] * 2, params["ngf"], 4, 2, 1, bias=False),
            nn.BatchNorm2d(params["ngf"]),
            nn.ReLU(True),
            # 特徴マップサイズ: (ngf) x 16 x 16
            nn.ConvTranspose2d(params["ngf"], params["nc"], 4, 2, 1, bias=False),
            nn.Tanh()
            # 出力画像サイズ: (nc) x 32 x 32
        )

    def forward(self, input):
        return self.main(input)

# インスタンス化
netG = Generator(params).to(device)

### 🛡️ 識別器（Discriminator）

識別器は、入力された画像が「本物（訓練データ）」か「偽物（Generatorが生成したもの）」かを判定するバイナリ分類器です。
CNN 画像分類器と同様の構成を持ちますが、DCGAN の設計指針に従い、全結合層の代わりに **畳み込み層（Convolution）** を重ねて情報を集約します。活性化関数には **LeakyReLU** を使用します。

U-Net が「どのピクセルが何であるか」を出力するのに対し、識別器は最終的に画像全体に対して「本物である確率」という **スカラー値** を出力します。

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 入力画像サイズ: (nc) x 32 x 32
            nn.Conv2d(params["nc"], params["ndf"], 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 特徴マップサイズ: (ndf) x 16 x 16
            nn.Conv2d(params["ndf"], params["ndf"] * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(params["ndf"] * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 特徴マップサイズ: (ndf*2) x 8 x 8
            nn.Conv2d(params["ndf"] * 2, params["ndf"] * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(params["ndf"] * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 特徴マップサイズ: (ndf*4) x 4 x 4
            nn.Conv2d(params["ndf"] * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# インスタンス化
netD = Discriminator(params).to(device)

## ⚖️ 学習の設定と重みの初期化（本題ではありません）

DCGAN の論文では、全てのネットワークの重みを平均 0、標準偏差 0.02 の正規分布で初期化することが推奨されています。また、損失関数には **バイナリエントロピー誤差（BCELoss）** を、最適化アルゴリズムには **Adam** を使用します。

In [ ]:
# 重みの初期化関数
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# 初期化の適用
netG.apply(weights_init)
netD.apply(weights_init)

# 損失関数
criterion = nn.BCELoss()

# 固定ノイズ（学習の進捗を同じ視点で確認するため）
fixed_noise = torch.randn(64, params["nz"], 1, 1, device=device)

# 最適化手法
optimizerD = optim.Adam(netD.parameters(), lr=params["lr"], betas=(params["beta1"], 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=params["lr"], betas=(params["beta1"], 0.999))

## 🚀 学習の実行

学習ループでは、以下の 2 ステップを繰り返します。
1. **識別器（D）の更新**: 本物を「本物（1）」、偽物を「偽物（0）」と正しく判定できるように学習します。
2. **生成器（G）の更新**: 生成した画像が D に「本物」と誤認される（D の出力を 1 に近づける）ように学習します。

In [ ]:
from tqdm.notebook import tqdm

img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
for epoch in range(params["n_epochs"]):
    progress_bar = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{params['n_epochs']}]", leave=False)
    for i, data in enumerate(progress_bar):

        # --- (1) 識別器 D の更新 ---
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 1.0, dtype=torch.float, device=device) # 本物ラベル = 1

        # 本物画像でのフォワード
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()

        # 偽物画像の生成とフォワード
        noise = torch.randn(b_size, params["nz"], 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0.0) # 偽物ラベル = 0
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()

        errD = errD_real + errD_fake
        optimizerD.step()

        # --- (2) 生成器 G の更新 ---
        netG.zero_grad()
        label.fill_(1.0) # 生成器にとっては、Dを騙して 1 と言わせたい
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

        # ログの記録
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        progress_bar.set_postfix({"Loss_D": f"{errD.item():.4f}", "Loss_G": f"{errG.item():.4f}"})

    # 一定期間ごとに生成画像を保存
    if (epoch % 10 == 0) or (epoch == params["n_epochs"]-1):
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu()
        img_list.append(make_grid(fake, padding=2, normalize=True))

# 損失の推移
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

print("Training Finished.")

## 📊 結果の可視化

学習後の損失関数の推移と、Generator が生成した画像を確認します。

In [ ]:
# 最後の_エポックで生成された画像を表示
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title(f"Generated Images for {params['target_class_name']}")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

if len(dataloader) > 0:
    real_batch = next(iter(dataloader))

    # --- 変更箇所 ---

    # 1行2列のサブプロットを作成 (figsizeを (15,15) から (16,8) のような横長に調整)
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))

    # 1. 順序を逆にする (Fake Images を左に [0])
    axes[0].imshow(np.transpose(img_list[-1], (1, 2, 0)))
    axes[0].set_title(f"Fake Images ({params['target_class_name']})")

    # 2. 順序を逆にする (Real Images を右に [1])
    # make_gridの処理を変数に格納
    real_images_grid = make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu()
    axes[1].imshow(np.transpose(real_images_grid, (1, 2, 0)))
    axes[1].set_title(f"Real Images ({params['target_class_name']})")

    # 2. 両方の枠線を調整 (両方の軸を非表示にする)
    axes[0].axis('off')
    axes[1].axis('off')

    # サブプロット間の余白を調整
    plt.tight_layout()

    plt.show()
    # --- 変更ここまで ---

else:
    print(f"Cannot display real images as the dataloader for {params['target_class_name']} is empty.")